라이브러리 임포트 및 시드고정

In [8]:
## 라이브러리 임포트, 시드고정
import os
import json 
import random
import numpy as np
from PIL import Image

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, models
from tqdm import tqdm
import copy

from sklearn.metrics import (
    accuracy_score, precision_score, recall_score,
    f1_score, confusion_matrix, classification_report
)

# ⭐️ Deterministic 연산을 위한 환경 변수 설정 (에러 해결) ⭐️
# 이 환경 변수를 설정해야 CUDA 10.2 이상에서 재현성이 보장됩니다.
os.environ['CUBLAS_WORKSPACE_CONFIG'] = ':4096:8'

# 시드 고정 함수
def set_seed(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
    
    # Deterministic 알고리즘 사용 설정 (이전 코드에서 유지됨)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False # True일 경우 Deterministic을 막을 수 있음

set_seed(42)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"✅ Device set to: {device}")

✅ Device set to: cuda


경로 및 하이퍼파라미터 설정

In [ ]:
# ★ 경로 설정 (사용자 환경에 맞게 수정) ★
IMG_ROOT = "/workspace/merge_data/data/img"

# 하이퍼파라미터
BATCH_SIZE = 32
NUM_WORKERS = 4
NUM_EPOCHS = 20
LEARNING_RATE = 0.0001

In [10]:
## 데이터 유효성 검증🔎
def validate_image_data(root_dir):
    print(f"🔍 데이터 검증 시작: {root_dir}")
    
    total_files = 0
    corrupt_files = 0
    
    valid_ext = ('.jpg', '.jpeg', '.png', '.bmp')
    
    for root, dirs, files in os.walk(root_dir):
        image_files = [f for f in files if f.lower().endswith(valid_ext)]
        
        if len(image_files) > 0:
            print(f"  📁 Folder: {os.path.basename(root)} | Images: {len(image_files)}")
            
            for img_file in tqdm(image_files, desc=f"Checking {os.path.basename(root)}", leave=False):
                img_path = os.path.join(root, img_file)
                try:
                    with Image.open(img_path) as img:
                        img.verify()
                except (IOError, SyntaxError) as e:
                    print(f"    ❌ Corrupt file found: {img_path}")
                    corrupt_files += 1
                
                total_files += 1

    print("-" * 40)
    print(f"✅ 검증 완료")
    print(f"   - 총 이미지 수: {total_files}")
    print(f"   - 손상된 파일 수: {corrupt_files}")
    
    if corrupt_files > 0:
        print("⚠️ 경고: 손상된 파일이 존재합니다. 해당 파일을 제거하거나 확인해주세요.")
    else:
        print("🎉 모든 파일이 정상입니다.")

# 검증 실행
if os.path.exists(IMG_ROOT):
    validate_image_data(IMG_ROOT)
else:
    print(f"❌ 경로 오류: {IMG_ROOT} 가 존재하지 않습니다.")

🔍 데이터 검증 시작: /workspace/merge_data_binary
  📁 Folder: other | Images: 852


Checking other:  18%|█▊        | 154/852 [00:00<00:01, 642.67it/s]

  📁 Folder: happy | Images: 299


  📁 Folder: other | Images: 4348


  📁 Folder: happy | Images: 5202


----------------------------------------
✅ 검증 완료
   - 총 이미지 수: 10701
   - 손상된 파일 수: 0
🎉 모든 파일이 정상입니다.


In [11]:
class EmotionDataset(Dataset):
    """
    JSON 파일을 사용하지 않고, 폴더 구조 (train/happy, val/anger 등) 만으로 
    이미지 경로와 레이블을 로딩하는 Dataset 클래스입니다.
    """
    def __init__(self, img_root, split="train", transform=None):
        self.img_root = os.path.join(img_root, split)
        self.transform = transform
        
        # 폴더 이름과 이진 레이블 매핑
        self.emotions = ["anger", "happy", "panic", "sadness"]
        self.label_map = {"happy": 1, "anger": 0, "panic": 0, "sadness": 0}
        self.samples = []
        
        valid_ext = ('.jpg', '.jpeg', '.png', '.bmp')

        print(f"\n[{split.upper()} Dataset Loading...]")
        
        # ⭐️ 폴더 구조를 직접 스캔하여 데이터 로딩 ⭐️
        for emo in self.emotions:
            emo_dir = os.path.join(self.img_root, emo)
            
            if not os.path.isdir(emo_dir):
                print(f"  [WARN] Folder not found: {emo_dir}")
                continue
                
            label = self.label_map[emo]
            count = 0
            
            for filename in os.listdir(emo_dir):
                if filename.lower().endswith(valid_ext):
                    img_path = os.path.join(emo_dir, filename)
                    self.samples.append((img_path, label))
                    count += 1
            
            # print(f"  - {emo}: {count} images loaded")

        random.shuffle(self.samples)
        print(f"  >> Total {split} samples: {len(self.samples)}")

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        img_path, label = self.samples[idx]
        
        try:
            image = Image.open(img_path).convert("RGB")
        except Exception as e:
            print(f"Error loading image {img_path}: {e}. Skipping to index 0.")
            return self.__getitem__(0)

        if self.transform is not None:
            image = self.transform(image)

        label = torch.tensor(label, dtype=torch.long)
        return image, label

In [13]:
# ResNet 기본 입력 크기 224x224 기준
train_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    # 정규화 유지 (모델 학습 안정성을 위한 필수 전처리)
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    ),
])

val_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    ),
])

# 데이터셋 생성
train_dataset = EmotionDataset(IMG_ROOT, split="train", transform=train_transform)
val_dataset   = EmotionDataset(IMG_ROOT, split="val",   transform=val_transform)

# 데이터로더 생성
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=NUM_WORKERS)
val_loader   = DataLoader(val_dataset,   batch_size=BATCH_SIZE, shuffle=False, num_workers=NUM_WORKERS)

print("\n✅ DataLoaders are ready.")


[NEW_DATA Dataset Loading...]
  [WARN] Folder not found: /workspace/merge_data_binary/new_data/anger
  [WARN] Folder not found: /workspace/merge_data_binary/new_data/happy
  [WARN] Folder not found: /workspace/merge_data_binary/new_data/panic
  [WARN] Folder not found: /workspace/merge_data_binary/new_data/sadness
  >> Total new_data samples: 0

[NEW_DATA_VAL Dataset Loading...]
  [WARN] Folder not found: /workspace/merge_data_binary/new_data_val/anger
  [WARN] Folder not found: /workspace/merge_data_binary/new_data_val/happy
  [WARN] Folder not found: /workspace/merge_data_binary/new_data_val/panic
  [WARN] Folder not found: /workspace/merge_data_binary/new_data_val/sadness
  >> Total new_data_val samples: 0


ValueError: num_samples should be a positive integer value, but got num_samples=0

In [ ]:
## CBAM 모듈 및 ResNetCBAM 모델 정의 (⭐️ CBAM 추가됨)CBAM 모듈 및 ResNetCBAM 모델 정의 (⭐️ CBAM 추가됨)
## CBAM 모듈과, ResNet에 CBAM을 통합한 ResNetCBAM 모델을 정의합니다.

print("🛠️ CBAM 모듈 및 ResNetCBAM 모델 설정 중...")

# ⭐️ 채널 어텐션 모듈 (Channel Attention Module) 정의
class ChannelAttention(nn.Module):
    def __init__(self, in_planes, ratio=16):
        super(ChannelAttention, self).__init__()
        self.avg_pool = nn.AdaptiveAvgPool2d(1)
        self.max_pool = nn.AdaptiveMaxPool2d(1)
        self.fc = nn.Sequential(nn.Conv2d(in_planes, in_planes // ratio, 1, bias=False),
                                nn.ReLU(),
                                nn.Conv2d(in_planes // ratio, in_planes, 1, bias=False))
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        avg_out = self.fc(self.avg_pool(x))
        max_out = self.fc(self.max_pool(x))
        out = avg_out + max_out
        return self.sigmoid(out)

# ⭐️ 공간 어텐션 모듈 (Spatial Attention Module) 정의
class SpatialAttention(nn.Module):
    def __init__(self, kernel_size=7):
        super(SpatialAttention, self).__init__()
        self.conv1 = nn.Conv2d(2, 1, kernel_size, padding=kernel_size//2, bias=False)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        avg_out = torch.mean(x, dim=1, keepdim=True)
        max_out, _ = torch.max(x, dim=1, keepdim=True)
        x_out = torch.cat([avg_out, max_out], dim=1)
        x_out = self.conv1(x_out)
        return self.sigmoid(x_out)

# ⭐️ CBAM (Channel and Spatial Attention Module) 정의
class CBAM(nn.Module):
    def __init__(self, in_planes, ratio=16, kernel_size=7):
        super(CBAM, self).__init__()
        self.ca = ChannelAttention(in_planes, ratio)
        self.sa = SpatialAttention(kernel_size)

    def forward(self, x):
        out = x * self.ca(x)
        out = out * self.sa(out)
        return out

# ⭐️ ResNet18 백본에 CBAM을 통합한 모델 정의
class ResNetCBAM(nn.Module):
    def __init__(self, num_classes=2):
        super(ResNetCBAM, self).__init__()
        
        # 1. 사전학습된 ResNet18 로드
        resnet = models.resnet18(weights=models.ResNet18_Weights.IMAGENET1K_V1)
        
        # ResNet의 초기 Conv1, bn1, relu, maxpool을 그대로 사용
        self.conv1 = resnet.conv1
        self.bn1 = resnet.bn1
        self.relu = resnet.relu
        self.maxpool = resnet.maxpool
        
        # ResNet의 각 Stage (layer1, layer2, layer3, layer4)를 가져옴
        self.layer1 = resnet.layer1
        self.layer2 = resnet.layer2
        self.layer3 = resnet.layer3
        self.layer4 = resnet.layer4
        
        # ⭐️ 각 Residual Block 이후 또는 Layer 이후에 CBAM 모듈 삽입 ⭐️
        # ResNet18의 각 레이어의 출력 채널에 맞춰 CBAM을 초기화합니다.
        #self.cbam1 = CBAM(64)  # layer1의 출력 채널 (ResNet18)
        self.cbam2 = CBAM(128) # layer2의 출력 채널
        self.cbam3 = CBAM(256) # layer3의 출력 채널
        #self.cbam4 = CBAM(512) # layer4의 출력 채널
        ## CBAM 모듈은 stage 2,3에만 적용
        
        self.avgpool = resnet.avgpool
        
        # 최종 분류기를 2클래스로 교체
        num_features = resnet.fc.in_features
        self.fc = nn.Linear(num_features, num_classes)

    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)
        
        x = self.layer1(x)
        x = self.cbam1(x) # ⭐️ CBAM 삽입 지점
        x = self.layer2(x)
        x = self.cbam2(x) # ⭐️ CBAM 삽입 지점
        x = self.layer3(x)
        x = self.cbam3(x) # ⭐️ CBAM 삽입 지점
        x = self.layer4(x)
        x = self.cbam4(x) # ⭐️ CBAM 삽입 지점
        
        x = self.avgpool(x)
        x = torch.flatten(x, 1)# ⭐️ ResNet18 백본에 CBAM을 Stage 2와 3에만 통합한 모델 정의
class ResNetCBAM(nn.Module):
    def __init__(self, num_classes=2):
        super(ResNetCBAM, self).__init__()
        
        # 1. 사전학습된 ResNet18 로드
        resnet = models.resnet18(weights=models.ResNet18_Weights.IMAGENET1K_V1)
        
        # ResNet의 초기 Conv1, bn1, relu, maxpool을 그대로 사용
        self.conv1 = resnet.conv1
        self.bn1 = resnet.bn1
        self.relu = resnet.relu
        self.maxpool = resnet.maxpool
        
        # ResNet의 각 Stage (layer1, layer2, layer3, layer4)를 가져옴
        self.layer1 = resnet.layer1
        self.layer2 = resnet.layer2
        self.layer3 = resnet.layer3
        self.layer4 = resnet.layer4
        
        # ⭐️ Stage 2와 Stage 3의 출력 채널에 맞춰 CBAM 초기화
        self.cbam2 = CBAM(128) # layer2의 출력 채널
        self.cbam3 = CBAM(256) # layer3의 출력 채널
        
        # self.cbam1과 self.cbam4는 정의하지 않음
        
        self.avgpool = resnet.avgpool
        
        # 최종 분류기를 2클래스로 교체
        num_features = resnet.fc.in_features
        self.fc = nn.Linear(num_features, num_classes)

    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)
        
        x = self.layer1(x)
        # x = self.cbam1(x) # ⭐️ Stage 1 CBAM 제거
        
        x = self.layer2(x)
        x = self.cbam2(x) # ⭐️ Stage 2 CBAM 유지
        
        x = self.layer3(x)
        x = self.cbam3(x) # ⭐️ Stage 3 CBAM 유지
        
        x = self.layer4(x)
        # x = self.cbam4(x) # ⭐️ Stage 4 CBAM 제거
        
        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.fc(x)
        return x

# 모델, 손실함수, 최적화함수 초기화
model = ResNetCBAM(num_classes=2).to(device) # ResNetCBAM 모델 사용
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE, weight_decay=1e-4)

print("✅ ResNetCBAM Model (CBAM @ Stage 2, 3) is ready!")

🛠️ CBAM 모듈 및 ResNetCBAM 모델 설정 중...


✅ ResNetCBAM Model (CBAM @ Stage 2, 3) is ready!


In [ ]:
## 학습 함수 정의 ## epochs 설정
def train_model(model, criterion, optimizer, train_loader, val_loader, num_epochs=10):
    best_model_wts = copy.deepcopy(model.state_dict())
    best_val_acc = 0.0

    for epoch in range(num_epochs):
        print(f"\nEpoch {epoch+1}/{num_epochs}")
        print("-" * 30)

        # ---- Train ----
        model.train()
        running_loss = 0.0
        running_corrects = 0
        total = 0

        for inputs, labels in tqdm(train_loader, desc="Training", leave=False):
            inputs = inputs.to(device)
            labels = labels.to(device)

            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)

            _, preds = torch.max(outputs, 1)

            loss.backward()
            optimizer.step()

            running_loss += loss.item() * inputs.size(0)
            running_corrects += torch.sum(preds == labels.data)
            total += labels.size(0)
        
        # ⭐️ 학습 후 GPU 캐시 메모리 정리
        if device.type == 'cuda':
            torch.cuda.empty_cache()

        epoch_loss = running_loss / total
        epoch_acc = running_corrects.double() / total
        print(f"Train Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}")

        # ---- Validation ----
        model.eval()
        val_loss = 0.0
        val_corrects = 0
        val_total = 0

        with torch.inference_mode():
            for inputs, labels in tqdm(val_loader, desc="Validation", leave=False):
                inputs = inputs.to(device)
                labels = labels.to(device)

                outputs = model(inputs)
                loss = criterion(outputs, labels)
                _, preds = torch.max(outputs, 1)

                val_loss += loss.item() * inputs.size(0)
                val_corrects += torch.sum(preds == labels.data)
                val_total += labels.size(0)
        
        # ⭐️ 검증 후 GPU 캐시 메모리 정리
        if device.type == 'cuda':
            torch.cuda.empty_cache()

        val_loss /= val_total
        val_acc = val_corrects.double() / val_total
        print(f"Val   Loss: {val_loss:.4f} Acc: {val_acc:.4f}")

        # Best Model 저장
        if val_acc > best_val_acc:
            best_val_acc = val_acc
            best_model_wts = copy.deepcopy(model.state_dict())
            torch.save(best_model_wts, "Res_CBAM_v2.pth") # ⭐️ 파일명 변경
            print(f"⭐ Best model saved! (Acc: {best_val_acc:.4f})")

    print(f"\nTraining Complete. Best Val Acc: {best_val_acc:.4f}")
    model.load_state_dict(best_model_wts)
    return model

In [ ]:
## 학습 실행
# 학습 시작
trained_model = train_model(model, criterion, optimizer, train_loader, val_loader, num_epochs=NUM_EPOCHS)


Epoch 1/20
------------------------------


Training:   0%|          | 0/183 [00:00<?, ?it/s]

Train Loss: 0.2858 Acc: 0.8778


Val   Loss: 0.2081 Acc: 0.9244
⭐ Best model saved! (Acc: 0.9244)

Epoch 2/20
------------------------------


Train Loss: 0.1000 Acc: 0.9617


...

Val   Loss: 0.2843 Acc: 0.9505
⭐ Best model saved! (Acc: 0.9505)

Epoch 19/20
------------------------------


Train Loss: 0.0116 Acc: 0.9959


Val   Loss: 0.2648 Acc: 0.9409

Epoch 20/20
------------------------------


Train Loss: 0.0171 Acc: 0.9950


Val   Loss: 0.2876 Acc: 0.9418

Training Complete. Best Val Acc: 0.9505


In [ ]:
## 성능 평가
print("\n📊 최종 성능 평가 중...")

trained_model.eval()
all_labels = []
all_preds = []

with torch.inference_mode():
    for inputs, labels in tqdm(val_loader, desc="Evaluating"):
        inputs = inputs.to(device)
        labels = labels.to(device)

        outputs = trained_model(inputs)
        _, preds = torch.max(outputs, 1)

        all_labels.extend(labels.cpu().numpy())
        all_preds.extend(preds.cpu().numpy())

all_labels = np.array(all_labels)
all_preds = np.array(all_preds)

# Metrics 계산
acc  = accuracy_score(all_labels, all_preds)
prec = precision_score(all_labels, all_preds, pos_label=1)
rec  = recall_score(all_labels, all_preds, pos_label=1)
f1   = f1_score(all_labels, all_preds, pos_label=1)
cm   = confusion_matrix(all_labels, all_preds)

print("\n=== Val Metrics (binary: 1=HAPPY, 0=OTHERS) ===")
print(f"Accuracy : {acc:.4f}")
print(f"Precision: {prec:.4f}")
print(f"Recall   : {rec:.4f}")
print(f"F1-score : {f1:.4f}")
print("\nConfusion Matrix (rows: true, cols: pred)")
print(cm)

print("\nClassification report:")
print(classification_report(all_labels, all_preds, target_names=["OTHERS", "HAPPY"]))


📊 최종 성능 평가 중...


Evaluating: 100%|██████████| 36/36 [00:28<00:00,  1.28it/s]


=== Val Metrics (binary: 1=HAPPY, 0=OTHERS) ===
Accuracy : 0.9505
Precision: 0.9261
Recall   : 0.8796
F1-score : 0.9022

Confusion Matrix (rows: true, cols: pred)
[[831  21]
 [ 36 263]]

Classification report:
              precision    recall  f1-score   support

      OTHERS       0.96      0.98      0.97       852
       HAPPY       0.93      0.88      0.90       299

    accuracy                           0.95      1151
   macro avg       0.94      0.93      0.93      1151
weighted avg       0.95      0.95      0.95      1151

